In [69]:
import gurobipy as grb
# https://www.youtube.com/watch?v=Copxhl2FShg

In [22]:
# Modelfurniture problem
f = grb.Model('Furniture')

In [23]:
# Variables 
x1 = f.addVar(name="chairs")
x2 = f.addVar(name="tables")

In [24]:
# Vamos a maximizar las ganancias por venta de mesas y sillas.
f.setObjective(45*x1 + 80*x2, GRB.MAXIMIZE)

In [25]:
# Restricciones para la producción basa en la disponibilidad de caoba
# La silla consume 5 y la mesa 20
f.addConstr(5*x1 +20*x2 <= 400,'mahogany')
#Restricción para las horas de trabajo necesarias
f.addConstr(10*x1 + 15*x2 <= 450, 'labor')

<gurobi.Constr *Awaiting Model Update*>

In [26]:
f.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xd0437183
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  Objective range  [5e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 5e+02]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5000000e+31   2.968750e+30   6.500000e+01      0s
       2    2.2000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  2.200000000e+03


In [27]:
# Mostrar resultados
for v in f.getVars():
    print(f"{v.varName}: {v.X}")
print('Ganancia óptima total: ',f.objVal)

chairs: 24.0
tables: 14.0
Ganancia óptima total:  2200.0


# No obstante, cuando contamos con muchas variables hay mejores maneras de ingresar los datos.

In [70]:
# Solución Parametrizada
resources, capacity = grb.multidict({
    'mahogany': 400,
    'labor': 450,
})
print(resources, capacity)

['mahogany', 'labor'] {'mahogany': 400, 'labor': 450}


In [71]:
# products, data
products, price = grb.multidict({
    'chair':45,
    'table': 80,
})
#products es un set, price es parámetro
print(products, price)

['chair', 'table'] {'chair': 45, 'table': 80}


In [72]:
# Bill of materials: recursos por producto
bom = {
    ('mahogany','chair'): 5,
    ('mahogany', 'table'): 20,
    ('labor', 'chair'): 10,
    ('labor', 'table'): 15,
}

In [73]:
f = grb.Model('Furniture')

In [74]:
make = f.addVars(products, name='make')

In [75]:
res = f.addConstrs(((sum(bom[r,p]*make[p] for p in products) <= capacity[r]) for r in resources),name='R')

In [76]:
f.setObjective(make.prod(price),GRB.MAXIMIZE)

In [80]:
f.write('furniture.lp')

In [78]:
f.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xd0437183
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  Objective range  [5e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 5e+02]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5000000e+31   2.968750e+30   6.500000e+01      0s
       2    2.2000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  2.200000000e+03


In [79]:
for v in f.getVars():   
    if(abs(v.x)>1e-6):
         print(f"{v.varName}: {v.X}")
print('Ganancia óptima total: ',f.objVal)

make[chair]: 24.0
make[table]: 14.0
Ganancia óptima total:  2200.0
